In [19]:
#Basics
import pandas as pd
import numpy as np
import itertools

#Time series 
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

# Ignoring warning messages from python
import warnings
warnings.filterwarnings('ignore')

#Modules
import wrangle as wr

<div class="alert alert-info"><b>Model Evaluation:  These models will be evalutated using root-mean-square errorr (RMSE) and mean absolute percentage error (MAPE) on a 1 day and 3 day basis.</b>
    <ul>
        <li><b>RMSE</b> allows us to see the overall error of the model in the energy load units (MW).</li>
        <li><b>MAPE</b> complements that by telling us how accurate the model is on a percentage basis.  This is important as the energy load varies greatly throughout the year, so a 1000 MW RMSE on a hot summer day will have a much lower MAPE percentage than a 1000 MW RMSE on a comfortable Saturday in the Fall.</li>
        <li>The <b>3 day metrics</b> will tell us the overall performance of the 3 day forecast.  However, we expect the 3rd day to be less accurate than the 1st day and production models will be re-run on a daily or sub-daily basis. For that reason, we also will look at <b>1 day metrics</b> to see the model performance over the first day only.</li>
    </ul><div>

<div class="alert alert-info"><b>Cross-Validation Notes:</b><br><br>Cross validation was performed on a sliding window.  Models were trained on <b>4 year initial windows</b>, with a <b>forecasting horizon of 3 days</b>.  Due to the large amount of data we had available, we minimized processing time by using a <b>36 day sliding offset</b>.  A 36 day offset ensured that our forecasting window started at different days of the week and rotated through different parts of the month. <br><br> The downside to this approach is that it is possible the model is not tested over more anomalous time periods (such as holidays or major weather events like Hurrican Harvey).  A smaller sliding offset (such as 1 or 2 days) would catch those time periods, however it would require much more processing time. </div>

### Prophet Models

**Note:** You will notice that we re-acquire the dataframes in this section.  Prophet requires time zone naive inputs, however we wanted our explore section above to display CST time zones for readability.

<div class="alert alert-danger"><b>Final Report Prep:  Make sure those two get prophet functions (seen in next cell) are in the main wrangle file.  I have not added them yet as I'm not sure if Greg has all his changes in there or not. - Cayt</b></div>

In [10]:
#Acquire data
dfr = wr.get_prophet_df_w_meantemp()
#Filter train based on CST time (EOY 2017, time zone naive UTC time is in this df)
trainr = dfr[dfr.ds < '2018-01-01 06:00:00']

#Create df w/o regression column
train = trainr[['ds','y']].copy()

In [11]:
#Set metrics to evaluate
metrics = ['rmse','mape']

##### Basic Prophet model
- Uses default Prophet settings:
  - Auto for all seasonality
  - No holidays
  - No regression

In [12]:
#Create and Fit Basic Prophet model
model = Prophet().fit(train)

#Create cross-validation matrix
df_cv = cross_validation(model,initial='1461 days', period='36 days', horizon = '3 days')

#Get 1 and 3 day model performance
df_p_1d = performance_metrics(df_cv,rolling_window=.33,metrics=metrics)
df_p_3d = performance_metrics(df_cv,rolling_window=1,metrics=metrics)

11:02:30 - cmdstanpy - INFO - Chain [1] start processing
11:02:55 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/41 [00:00<?, ?it/s]

11:02:58 - cmdstanpy - INFO - Chain [1] start processing
11:03:08 - cmdstanpy - INFO - Chain [1] done processing
11:03:09 - cmdstanpy - INFO - Chain [1] start processing
11:03:17 - cmdstanpy - INFO - Chain [1] done processing
11:03:19 - cmdstanpy - INFO - Chain [1] start processing
11:03:30 - cmdstanpy - INFO - Chain [1] done processing
11:03:31 - cmdstanpy - INFO - Chain [1] start processing
11:03:44 - cmdstanpy - INFO - Chain [1] done processing
11:03:45 - cmdstanpy - INFO - Chain [1] start processing
11:03:57 - cmdstanpy - INFO - Chain [1] done processing
11:03:58 - cmdstanpy - INFO - Chain [1] start processing
11:04:10 - cmdstanpy - INFO - Chain [1] done processing
11:04:11 - cmdstanpy - INFO - Chain [1] start processing
11:04:21 - cmdstanpy - INFO - Chain [1] done processing
11:04:22 - cmdstanpy - INFO - Chain [1] start processing
11:04:35 - cmdstanpy - INFO - Chain [1] done processing
11:04:37 - cmdstanpy - INFO - Chain [1] start processing
11:04:49 - cmdstanpy - INFO - Chain [1]

In [14]:
#print model results
wr.print_model_results('Basic Prophet',df_p_1d,df_p_3d)


Basic Prophet model performance:
1 day rmse: 1220.0 MW
1 day mape: 8.1%
3 day rmse: 1302.0 MW
3 day mape: 8.5%


<div class="alert alert-danger"><b>Final Report Prep:  Need to add this new function to wrangle, then update the above line to reference wrangle.</b></div>

##### Prophet model v2 - Hyperparameter first pass 
- Remove auto-seasonality
- manually add seasons:
  - daily fourier order of 6, with a scale of 30
  - weekly fourier order of 6, with a scale of 30
  - yearly fourier order of 6, with a scale of 10
- Added US Federal observed holidays
- Added mean temperature regressor

In [20]:
#PREP Holidays
#create calendar object
cal = USFederalHolidayCalendar()
#get holidays as list of dates
train_holidays = cal.holidays(start=trainr.ds.min(),end=trainr.ds.max())

# Transition to dataframe with holiday, ds columns
holiday_df = pd.DataFrame(trainr.ds)
#For each datetime, get if it lands on a holiday
holiday_df['holiday'] = holiday_df.ds.dt.date.astype(str).isin(train_holidays.astype(str)).astype(int)

In [21]:
#Filter down to just the holidays
only_holidays = holiday_df[holiday_df.holiday==1]
#Convert that column to string
only_holidays.holiday = only_holidays.holiday.astype(str)

In [ ]:
#Create the model
m = Prophet(yearly_seasonality=False,
            weekly_seasonality=False,
            daily_seasonality=False,
            holidays=only_holidays)
#Add seasonalities
m = m.add_seasonality(name='daily', 
                      period=1, 
                      fourier_order=6,
                      prior_scale=30
                     )
m = m.add_seasonality(name='weekly', 
                      period=7, 
                      fourier_order=6,
                      prior_scale=30
                     )
m = m.add_seasonality(name='yearly', 
                      period=365.25, 
                      fourier_order=6,
                      prior_scale=10
                     )
#Add Regressors
m = m.add_regressor('mean_temp')

In [15]:
model.seasonalities

OrderedDict([('yearly',
              {'period': 365.25,
               'fourier_order': 10,
               'prior_scale': 10.0,
               'mode': 'additive',
               'condition_name': None}),
             ('weekly',
              {'period': 7,
               'fourier_order': 3,
               'prior_scale': 10.0,
               'mode': 'additive',
               'condition_name': None}),
             ('daily',
              {'period': 1,
               'fourier_order': 4,
               'prior_scale': 10.0,
               'mode': 'additive',
               'condition_name': None})])